In [24]:
import pandas as pd
import os
import re
from io import StringIO

In [25]:
df = pd.read_csv("labels/Anno_coarse/list_bbox.txt", delim_whitespace=True, header=1)
df.head()

C:\Users\elise\AppData\Local\Temp\ipykernel_12352\1090048282.py:1: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df = pd.read_csv("labels/Anno_coarse/list_bbox.txt", delim_whitespace=True, header=1)


,image_name,x_1,y_1,x_2,y_2
0,img/Sheer_Pleated-Front_Blouse/img_00000001.jpg,72,79,232,273
1,img/Sheer_Pleated-Front_Blouse/img_00000002.jpg,67,59,155,161
2,img/Sheer_Pleated-Front_Blouse/img_00000003.jpg,65,65,156,200
3,img/Sheer_Pleated-Front_Blouse/img_00000004.jpg,51,62,167,182
4,img/Sheer_Pleated-Front_Blouse/img_00000005.jpg,46,88,166,262


In [26]:
file_path = 'labels/Anno_coarse/list_attr_cloth.txt'

transformed_lines = []
with open(file_path, 'r') as file:
    for line in file:
        transformed_line = re.sub(r' (?=\S)(?!\d)', '_', line.strip())
        transformed_line = transformed_line.title()
        transformed_lines.append(transformed_line)

data = '\n'.join(transformed_lines)
data = StringIO(data)

raw_attributes = pd.read_csv(data, sep='\s+', header=1)
column_names = ['attribute_name', 'attribute_type']
raw_attributes.columns = column_names
raw_attributes.head(10)

<>:13: SyntaxWarning: invalid escape sequence '\s'
<>:13: SyntaxWarning: invalid escape sequence '\s'
C:\Users\elise\AppData\Local\Temp\ipykernel_12352\2376651624.py:13: SyntaxWarning: invalid escape sequence '\s'
  raw_attributes = pd.read_csv(data, sep='\s+', header=1)


,attribute_name,attribute_type
0,A-Line,3
1,Abstract,1
2,Abstract_Chevron,1
3,Abstract_Chevron_Print,1
4,Abstract_Diamond,1
5,Abstract_Floral,1
6,Abstract_Floral_Print,1
7,Abstract_Geo,1
8,Abstract_Geo_Print,1
9,Abstract_Paisley,1


In [27]:
count_values = []
for row in raw_attributes.itertuples():
    attribute_name = getattr(row, 'attribute_name')
    attribute_type = getattr(row, 'attribute_type')
    count = df['image_name'].str.contains(attribute_name, regex=True).sum()  # Counts the number of True values
    count_values.append([attribute_name, attribute_type, count])  # Store texture and its count

count_df = pd.DataFrame(count_values, columns=['attribute_name', 'attribute_type', 'count'])
count_df

,attribute_name,attribute_type,count
0,A-Line,3,3252
1,Abstract,1,10496
2,Abstract_Chevron,1,231
3,Abstract_Chevron_Print,1,110
4,Abstract_Diamond,1,151
...,...,...,...
995,Zip-Pocket,4,139
996,Zip-Up,4,291
997,Zipped,4,83
998,Zipper,4,3465


In [28]:
selected_rows = []  # A list to hold the rows that meet the condition
for row in count_df.itertuples():
    if row.count > 1000:
        selected_rows.append(row)  # Append the row as a dictionary

attribute = pd.DataFrame(selected_rows)
attribute.drop(columns=['Index'], inplace=True)
attribute

,attribute_name,attribute_type,count
0,A-Line,3,3252
1,Abstract,1,10496
2,Abstract_Print,1,4239
3,Acid,2,1442
4,Acid_Wash,2,1415
...,...,...,...
187,Workout,5,1130
188,Woven,2,5687
189,Zip,4,4996
190,Zipper,4,3465


In [72]:
texture = attribute[attribute['attribute_type'] == 1]
fabric = attribute[attribute['attribute_type'] == 2]
shape = attribute[attribute['attribute_type'] == 3]
part = attribute[attribute['attribute_type'] == 4]
style = attribute[attribute['attribute_type'] == 5]

In [73]:
texture = texture.reset_index(drop=True) # Reset the index
texture = texture.reset_index(drop=False) # Display the index
texture = texture.drop(columns=['count','attribute_type'])
texture.head()

,index,attribute_name
0,0,Abstract
1,1,Abstract_Print
2,2,Baroque
3,3,Chevron
4,4,Colorblock


In [74]:
fabric = fabric.reset_index(drop=True) # Reset the index
fabric = fabric.reset_index(drop=False) # Display the index
fabric = fabric.drop(columns=['count','attribute_type'])
fabric.head()

,index,attribute_name
0,0,Acid
1,1,Acid_Wash
2,2,Bead
3,3,Beaded
4,4,Burnout


In [75]:
shape = shape.reset_index(drop=True) # Reset the index
shape = shape.reset_index(drop=False) # Display the index
shape = shape.drop(columns=['count','attribute_type'])
shape.head()

,index,attribute_name
0,0,A-Line
1,1,Asymmetric
2,2,Bodycon
3,3,Box
4,4,Boxy


In [76]:
part = part.reset_index(drop=True) # Reset the index
part = part.reset_index(drop=False) # Display the index
part = part.drop(columns=['count','attribute_type'])
part.head()

,index,attribute_name
0,0,Belted
1,1,Bow
2,2,Button
3,3,Buttoned
4,4,Collar


In [77]:
style = style.reset_index(drop=True) # Reset the index
style = style.reset_index(drop=False) # Display the index
style = style.drop(columns=['count','attribute_type']) 
style.head()

,index,attribute_name
0,0,Athletic
1,1,Babydoll
2,2,Baseball
3,3,Boyfriend
4,4,Classic


In [78]:
def csv_to_txt(csv_path, txt_path):
    with open(csv_path, 'r') as csv_file, open(txt_path, 'w') as txt_file:
        for line in csv_file:
            txt_file.write(line)